In [ ]:
import tiktoken
import pandas as pd


df = pd.read_csv("../data/results/test_all.csv")
encoding = tiktoken.get_encoding("cl100k_base")


input_tokens = []
output_tokens = []

for x in df.to_dict("records"):
    num_input_tokens = len(encoding.encode(x["inputs"]))
    num_output_tokens = len(encoding.encode(x["outputs"]))
    input_tokens.append(num_input_tokens)
    output_tokens.append(num_output_tokens)


input_costs = []
output_costs = []

for tokens in input_tokens:
    input_costs.append(round(tokens * 0.0005 / 1000, 4))

for tokens in output_tokens:
    output_costs.append(round(tokens *  0.0015 / 1000, 4))

print("Avg. input token: ", sum(input_tokens)/len(input_tokens))
print("Avg. output token costs: ", sum(output_tokens)/len(output_tokens))


In [ ]:
from data import Dependency
from prompt_settings import CfgNetPromptSettings, PrompSettingsFactory


dep = Dependency(
    project="piggymetrics",
    dependency_category="value-equality",
    option_name="EXPOSE",
    option_value="8080",
    option_type="PORT",
    option_file="Dockerfile",
    option_technology="Docker",
    dependent_option_name="server.port",
    dependent_option_value="8080",
    dependent_option_file="application.yml",
    dependent_option_type="PORT",
    dependent_option_technology="Spring-Boot"
)


prompt_settings = PrompSettingsFactory.get_prompt_settings(tool_name="cfgnet")

prompt_settings.get_format_prompt()

In [2]:
from cval import CVal
from data import Dependency

CONFIG_FILE = "../config.toml"
ENV_FILE = "../.env"
EVAL_DATA_DIR = "../data/evaluation/data"
INDEX_NAME = "tech-docs"

dep = Dependency(
    project="piggymetrics",
    dependency_category="value-equality",
    option_name="EXPOSE",
    option_value="8080",
    option_type="PORT",
    option_file="Dockerfile",
    option_technology="Docker",
    dependent_option_name="server.port",
    dependent_option_value="8080",
    dependent_option_file="application.yml",
    dependent_option_type="PORT",
    dependent_option_technology="Spring-Boot"
)

cval = CVal.init(
    config_file=CONFIG_FILE,
    env_file=ENV_FILE
)

response = cval.query(dependency=dep, index_name=INDEX_NAME)

[07/03/24 09:11:41] INFO     Ingestion engine initialized.                                          ]8;id=917706;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=333418;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py#39\39]8;;\

[07/03/24 09:11:42] INFO     Retrieval engine initialized.                                          ]8;id=865629;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=403028;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#28\28]8;;\

                    INFO     Ingestion engine initialized.                                          ]8;id=597280;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=125006;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py#39\39]8;;\

                    INFO     Ollama (llama3:8b) generator initialized.                              ]8;id=834533;file:///Users/sebastiansimon/GitHub/cval/src/generator.py\generator.py]8;;\:]8;id=30765;file:///Users/sebastiansimon/GitHub/cval/src/generator.py#84\84]8;;\

                    INFO     Initialize CfgNet prompt settings.                              ]8;id=626605;file:///Users/sebastiansimon/GitHub/cval/src/prompt_settings.py\prompt_settings.py]8;;\:]8;id=789583;file:///Users/sebastiansimon/GitHub/cval/src/prompt_settings.py#192\192]8;;\

...

[07/03/24 09:11:44] INFO     PyTorch version 2.3.1 available.                                          ]8;id=210542;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/datasets/config.py\config.py]8;;\:]8;id=526368;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/datasets/config.py#58\58]8;;\

/Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


[07/03/24 09:11:47] INFO     Rerank 10 retrieved nodes into 3 nodes.                                ]8;id=40365;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=622396;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#93\93]8;;\

                    INFO     Query llama3:8b                                                            ]8;id=476021;file:///Users/sebastiansimon/GitHub/cval/src/cval.py\cval.py]8;;\:]8;id=725861;file:///Users/sebastiansimon/GitHub/cval/src/cval.py#148\148]8;;\

[07/03/24 09:12:02] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=140134;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=310657;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\

In [3]:
response.response

'{ \n"plan": "Evaluate the EXPOSE configuration option in Dockerfile and compare it with the server.port configuration option in application.yml to determine whether they depend on each other based on value-equality.", \n"rationale": "The EXPOSE configuration option sets the port for exposure, while the server.port configuration option sets the port for the Spring Boot application. Since both options have the same value (8080), there is a possibility that they might be dependent on each other. However, without further information about how these configurations are used in the Docker container and the Spring Boot application, it\'s difficult to say for certain whether this dependency exists.", \n"uncertainty": 6, \n"isDependency": false \n}'